In [2]:
import urllib, urllib.request, xmltodict, feedparser
from datetime import datetime
from time import mktime
import pprint

import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def get_title_abs_arxiv(url):
    if 'arxiv' not in url: return -1,{}
    if 'pdf' in url: arxiv_id=''.join(url.split('pdf')[-2:-1])[1:-1]
    else: arxiv_id=url.split('/')[-1]
    query_url='http://export.arxiv.org/api/query?id_list='+arxiv_id
    data_dict = xmltodict.parse(urllib.request.urlopen(query_url).read())['feed']
    return {'title':data_dict['entry']['title'],'abstract':' '.join(data_dict['entry']['summary'].split('\n'))}

In [3]:
def fetch_daily_arxiv_papers(cat='cs.LG'):
    feedurl=f'https://rss.arxiv.org/rss/{cat}'
    feed=feedparser.parse(feedurl)
    datestr=datetime.fromtimestamp(mktime(feed['feed']['published_parsed'])).strftime('%d-%b-%Y')
    daily_papers=[]
    for e in feed['entries']:
        paper_entry={'id':e['id'],'title':e['title'],
                     'abstract':' '.join(e['summary'].split('\n')),
                     'date':datestr}
        daily_papers.append(paper_entry)
    return daily_papers

In [4]:
df = pd.read_csv('./Datasets/papers_of_interest/papers_of_interest.csv')
df['url'][0]

'https://arxiv.org/pdf/2209.11142.pdf'

In [5]:
get_title_abs_arxiv(df['url'][0])

{'title': 'A Generalist Neural Algorithmic Learner',
 'abstract': 'The cornerstone of neural algorithmic reasoning is the ability to solve algorithmic tasks, especially in a way that generalises out of distribution. While recent years have seen a surge in methodological improvements in this area, they mostly focused on building specialist models. Specialist models are capable of learning to neurally execute either only one algorithm or a collection of algorithms with identical control-flow backbone. Here, instead, we focus on constructing a generalist neural algorithmic learner -- a single graph neural network processor capable of learning to execute a wide range of algorithms, such as sorting, searching, dynamic programming, path-finding and geometry. We leverage the CLRS benchmark to empirically show that, much like recent successes in the domain of perception, generalist algorithmic learners can be built by "incorporating" knowledge. That is, it is possible to effectively learn algo

In [6]:
df.shape

(313, 2)

In [32]:
#BERT

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

title_embeddings_list = []
abstract_embeddings_list = []

for paper in range(df.shape[0]):
    # print(df['url'][paper])
    print(paper, end=' ')
    paper_data = get_title_abs_arxiv(df['url'][paper])
    for mode in ['title', 'abstract']:
        inputs = tokenizer(paper_data[mode], return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        pooled_embeddings = torch.mean(embeddings, dim=1)
        if mode == 'title':
            title_embeddings_list.append(pooled_embeddings.cpu().numpy())
        elif mode == 'abstract':
            abstract_embeddings_list.append(pooled_embeddings.cpu().numpy())

print()

# Convert lists of NumPy arrays to NumPy arrays
title_embeddings_np = np.vstack(title_embeddings_list)
abstract_embeddings_np = np.vstack(abstract_embeddings_list)

# Create DataFrame
df_embeddings = pd.DataFrame({
    'url' : df['url'].values.tolist(),
    'title_embeddings': [torch.tensor(embedding) for embedding in title_embeddings_list],
    'abstract_embeddings': [torch.tensor(embedding) for embedding in abstract_embeddings_list]
})

df_embeddings.to_pickle("./Datasets/papers_of_interest/bert_embeddings.pkl")


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [42]:
#SCIBERT

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased').to(device)

title_embeddings_list = []
abstract_embeddings_list = []

for paper in range(df.shape[0]):
    # print(df['url'][paper])
    print(paper, end=' ')
    paper_data = get_title_abs_arxiv(df['url'][paper])
    for mode in ['title', 'abstract']:
        inputs = tokenizer(paper_data[mode], return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        pooled_embeddings = torch.mean(embeddings, dim=1)
        if mode == 'title':
            title_embeddings_list.append(pooled_embeddings.cpu().numpy())
        elif mode == 'abstract':
            abstract_embeddings_list.append(pooled_embeddings.cpu().numpy())

print()

# Convert lists of NumPy arrays to NumPy arrays
title_embeddings_np = np.vstack(title_embeddings_list)
abstract_embeddings_np = np.vstack(abstract_embeddings_list)

# Create DataFrame
df_embeddings = pd.DataFrame({
    'url' : df['url'].values.tolist(),
    'title_embeddings': [torch.tensor(embedding) for embedding in title_embeddings_list],
    'abstract_embeddings': [torch.tensor(embedding) for embedding in abstract_embeddings_list]
})

df_embeddings.to_pickle("./Datasets/papers_of_interest/scibert_embeddings.pkl")



/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


0 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [45]:
#SPECTER

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('allenai/specter')
model = BertModel.from_pretrained('allenai/specter').to(device)

title_embeddings_list = []
abstract_embeddings_list = []

for paper in range(df.shape[0]):
    # print(df['url'][paper])
    print(paper, end=' ')
    paper_data = get_title_abs_arxiv(df['url'][paper])
    for mode in ['title', 'abstract']:
        inputs = tokenizer(paper_data[mode], return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        pooled_embeddings = torch.mean(embeddings, dim=1)
        if mode == 'title':
            title_embeddings_list.append(pooled_embeddings.cpu().numpy())
        elif mode == 'abstract':
            abstract_embeddings_list.append(pooled_embeddings.cpu().numpy())

print()

# Convert lists of NumPy arrays to NumPy arrays
title_embeddings_np = np.vstack(title_embeddings_list)
abstract_embeddings_np = np.vstack(abstract_embeddings_list)

# Create DataFrame
df_embeddings = pd.DataFrame({
    'url' : df['url'].values.tolist(),
    'title_embeddings': [torch.tensor(embedding) for embedding in title_embeddings_list],
    'abstract_embeddings': [torch.tensor(embedding) for embedding in abstract_embeddings_list]
})

df_embeddings.to_pickle("./Datasets/papers_of_interest/specter_embeddings.pkl")



/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


0 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [46]:
from transformers import AutoTokenizer, AutoModel

# load model and tokenizer
tokenizer1 = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
tokenizer3 = AutoTokenizer.from_pretrained('allenai/specter')
model1 = AutoModel.from_pretrained('bert-base-uncased')
model2 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
model3 = AutoModel.from_pretrained('allenai/specter')

papers = [ get_title_abs_arxiv(url) for url in df['url'] ]

# concatenate title and abstract
title_abs = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in papers]
# preprocess the input
inputs1 = tokenizer1(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs2 = tokenizer2(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs3 = tokenizer3(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
result1 = model1(**inputs1)
result2 = model2(**inputs2)
result3 = model3(**inputs3)
# take the first token in the batch as the embedding
embeddings1 = result1.last_hidden_state[:, 0, :]
embeddings2 = result2.last_hidden_state[:, 0, :]
embeddings3 = result3.last_hidden_state[:, 0, :]

df_embeddings = pd.DataFrame({
    'url' : df['url'].values.tolist(),
    'bert_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings1],
    'scibert_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings2],
    'specter_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings3]
})

df_embeddings.to_pickle("./Datasets/papers_of_interest/cat_embeddings.pkl")

/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/tmp/ipykernel_3338212/667730810.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'bert_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings1],
/tmp/ipykernel_3338212/667730810.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'scibert_cat_embedd

In [5]:
# Load CSV file into a DataFrame
df = pd.read_csv('./Datasets/daily_papers/daily_arxiv_papers.csv')
print(len(df))
# Keep rows where 'id' column contains 'arxiv'
df = df[df['id'].str.lower().str.contains('arxiv')]
# Reset index if needed
df.reset_index(drop=True, inplace=True)
print(len(df))


1173
1158


In [6]:
df

,id,title,abstract,date
0,oai:arXiv.org:2403.19669v1,Analyzing the Roles of Language and Vision in ...,arXiv:2403.19669v1 Announce Type: new Abstrac...,01-Apr-2024
1,oai:arXiv.org:2403.19717v1,A Picture is Worth 500 Labels: A Case Study of...,arXiv:2403.19717v1 Announce Type: new Abstrac...,01-Apr-2024
2,oai:arXiv.org:2403.19721v1,Computationally and Memory-Efficient Robust Pr...,arXiv:2403.19721v1 Announce Type: new Abstrac...,01-Apr-2024
3,oai:arXiv.org:2403.19792v1,MAPL: Model Agnostic Peer-to-peer Learning,arXiv:2403.19792v1 Announce Type: new Abstrac...,01-Apr-2024
4,oai:arXiv.org:2403.19800v1,Gegenbauer Graph Neural Networks for Time-vary...,arXiv:2403.19800v1 Announce Type: new Abstrac...,01-Apr-2024
...,...,...,...,...
1153,http://arxiv.org/abs/2404.17126v1,Deep Evidential Learning for Dose Prediction,"In this work, we present a novel application o...",2024-04-26
1154,http://arxiv.org/abs/2404.17120v1,Talking Nonsense: Probing Large Language Model...,Large language models (LLMs) exhibit excellent...,2024-04-26
1155,http://arxiv.org/abs/2404.17113v1,"MER 2024: Semi-Supervised Learning, Noise Robu...",Multimodal emotion recognition is an important...,2024-04-26
1156,http://arxiv.org/abs/2404.17110v1,Software Vulnerability Prediction in Low-Resou...,Background: Software Vulnerability (SV) predic...,2024-04-26


In [25]:
#BERT

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

title_embeddings_list = []
abstract_embeddings_list = []

for paper in range(df.shape[0]):
    # print(df['url'][paper])
    print(paper, end=' ')
    paper_data = {'title': df['title'][paper], 'abstract': df['abstract'][paper]}
    for mode in ['title', 'abstract']:
        inputs = tokenizer(paper_data[mode], return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        pooled_embeddings = torch.mean(embeddings, dim=1)
        if mode == 'title':
            title_embeddings_list.append(pooled_embeddings.cpu().numpy())
        elif mode == 'abstract':
            abstract_embeddings_list.append(pooled_embeddings.cpu().numpy())

print()

# Convert lists of NumPy arrays to NumPy arrays
title_embeddings_np = np.vstack(title_embeddings_list)
abstract_embeddings_np = np.vstack(abstract_embeddings_list)

# Create DataFrame
df_embeddings = pd.DataFrame({
    'id' : df['id'].values.tolist(),
    'title_embeddings': [torch.tensor(embedding) for embedding in title_embeddings_list],
    'abstract_embeddings': [torch.tensor(embedding) for embedding in abstract_embeddings_list]
})

df_embeddings.to_pickle("./Datasets/daily_papers/bert_embeddings.pkl")


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [26]:
#SCIBERT

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased').to(device)

title_embeddings_list = []
abstract_embeddings_list = []

for paper in range(df.shape[0]):
    # print(df['url'][paper])
    print(paper, end=' ')
    paper_data = {'title': df['title'][paper], 'abstract': df['abstract'][paper]}
    for mode in ['title', 'abstract']:
        inputs = tokenizer(paper_data[mode], return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        pooled_embeddings = torch.mean(embeddings, dim=1)
        if mode == 'title':
            title_embeddings_list.append(pooled_embeddings.cpu().numpy())
        elif mode == 'abstract':
            abstract_embeddings_list.append(pooled_embeddings.cpu().numpy())

print()

# Convert lists of NumPy arrays to NumPy arrays
title_embeddings_np = np.vstack(title_embeddings_list)
abstract_embeddings_np = np.vstack(abstract_embeddings_list)

# Create DataFrame
df_embeddings = pd.DataFrame({
    'id' : df['id'].values.tolist(),
    'title_embeddings': [torch.tensor(embedding) for embedding in title_embeddings_list],
    'abstract_embeddings': [torch.tensor(embedding) for embedding in abstract_embeddings_list]
})

df_embeddings.to_pickle("./Datasets/daily_papers/scibert_embeddings.pkl")


/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [27]:
#SPECTER

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('allenai/specter')
model = BertModel.from_pretrained('allenai/specter').to(device)

title_embeddings_list = []
abstract_embeddings_list = []

for paper in range(df.shape[0]):
    # print(df['url'][paper])
    print(paper, end=' ')
    paper_data = {'title': df['title'][paper], 'abstract': df['abstract'][paper]}
    for mode in ['title', 'abstract']:
        inputs = tokenizer(paper_data[mode], return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        pooled_embeddings = torch.mean(embeddings, dim=1)
        if mode == 'title':
            title_embeddings_list.append(pooled_embeddings.cpu().numpy())
        elif mode == 'abstract':
            abstract_embeddings_list.append(pooled_embeddings.cpu().numpy())

print()

# Convert lists of NumPy arrays to NumPy arrays
title_embeddings_np = np.vstack(title_embeddings_list)
abstract_embeddings_np = np.vstack(abstract_embeddings_list)

# Create DataFrame
df_embeddings = pd.DataFrame({
    'id' : df['id'].values.tolist(),
    'title_embeddings': [torch.tensor(embedding) for embedding in title_embeddings_list],
    'abstract_embeddings': [torch.tensor(embedding) for embedding in abstract_embeddings_list]
})

df_embeddings.to_pickle("./Datasets/daily_papers/specter_embeddings.pkl")


/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [7]:
from transformers import AutoTokenizer, AutoModel

# load model and tokenizer
tokenizer1 = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
tokenizer3 = AutoTokenizer.from_pretrained('allenai/specter')
model1 = AutoModel.from_pretrained('bert-base-uncased')
model2 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
model3 = AutoModel.from_pretrained('allenai/specter')

papers = []

for paper in range(df.shape[0]):
    papers.append({'title': df['title'][paper], 'abstract': df['abstract'][paper]})

# concatenate title and abstract
title_abs = [d['title'] + tokenizer1.sep_token + (d.get('abstract') or '') for d in papers]

#all three tokenizers have the same sep token
print("here")
# preprocess the input
inputs1 = tokenizer1(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs2 = tokenizer2(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs3 = tokenizer3(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
result1 = model1(**inputs1)
result2 = model2(**inputs2)
result3 = model3(**inputs3)
# take the first token in the batch as the embedding
embeddings1 = result1.last_hidden_state[:, 0, :]
embeddings2 = result2.last_hidden_state[:, 0, :]
embeddings3 = result3.last_hidden_state[:, 0, :]

df_embeddings = pd.DataFrame({
    'id' : df['id'].values.tolist(),
    'bert_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings1],
    'scibert_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings2],
    'specter_cat_embeddings': [torch.tensor(embedding) for embedding in embeddings3]
})

df_embeddings.to_pickle("./Datasets/daily_papers/cat_embeddings.pkl")



/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


here
